- 使用dataset(roberthsu2003/data_for_RAG)
- 使用模型(使用Microsoft開源的intfloat/multilingual-e5-large)
- 模型大約要2.24GB

In [ ]:
import torch
import numpy as np
from transformers import AutoTokenizer, AutoModel
import pandas as pd

class DocumentSearch:
    def __init__(self):
        self.tokenizer = AutoTokenizer.from_pretrained('intfloat/multilingual-e5-large')
        self.model = AutoModel.from_pretrained('intfloat/multilingual-e5-large')
        self.device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        self.model.to(self.device)

    def get_embedding(self, text, is_query=False):
        #增加前綴文字
        if is_query:
            text = f"query:{text}"
        else:
            text = f"passage:{text}"
        
        #編碼文本
        inputs = self.tokenizer(text, padding=True, truncation=True, max_length=512, return_tensors='pt')
        inputs = {k: v.to(self.device) for k, v in inputs.items()}
        

        #獲取嵌入向量
        with torch.no_grad():
            outputs = self.model(**inputs)
            #如果依據測試檢索能力的寫法是
            #embeddings = outputs.last_hidden_state.mean(dim=1)
            #但這種方法會比較好,看下面的說明1,2
            embeddings = outputs.last_hidden_state[:, 0, :] #使用[CLS]token的輸出#[CLS]標記通常被用來表示整個句子的語義
            embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)
        return embeddings.cpu().numpy()[0] #請看說明3

    def create_document_embeddings(self, df):
        """為輸入的dataFrame建立嵌入向量"""
        embeddings = []
        for _, row in df.iterrows():
            #組合Title和Text欄位
            text = f"Title:{row['Title']}\nText:{row['Text']}"
            embedding = self.get_embedding(text, is_query=False)
            embeddings.append(embedding)
        return embeddings
    
    def find_best_passage(self, query, df):
        """查載最相關的文本"""
        query_embedding = self.get_embedding(query, is_query=True)

        #計算相似度
        similarities = np.dot(np.stack(df['Embeddings'].values), query_embedding)#請看說明4

        #傳回最相關的文檔
        best_idx = np.argmax(similarities)
        return df.iloc[best_idx]['Text'], similarities[best_idx]
        
        
    

In [48]:
if __name__ == '__main__':
    #初始化文檔搜尋
    doc_search = DocumentSearch()

    # 準備數據
    documents = [
        {
            "Title": "操作氣候控制系統",
            "Text": "您的 Googlecar 配備氣候控制系統，可讓您調節車內的溫度和氣流。若要操作氣候控制系統，請使用中央控制台上的按鈕和旋鈕。"
        },
        {
            "Title": "觸控螢幕",
            "Text": "您的 Googlecar 擁有大型觸控螢幕顯示屏，可使用各種功能，包括導航、娛樂和氣候控制。"
        }
    ]

    # 建立DataFrame
    df = pd.DataFrame(documents)

    # 生成嵌入向量
    df['Embeddings'] = doc_search.create_document_embeddings(df)

    # 測試查詢
    query = "如何控制氣候?"

    best_text, similarity = doc_search.find_best_passage(query, df)
    print(f"Query:{query}")
    print(f"最相關的文字是(相似度:{similarity:.3f}):")
    print(best_text)




Query:如何控制氣候?
最相關的文字是(相似度:0.885):
您的 Googlecar 配備氣候控制系統，可讓您調節車內的溫度和氣流。若要操作氣候控制系統，請使用中央控制台上的按鈕和旋鈕。


## 說明1

1. `embeddings = outputs.last_hidden_state[:, 0, :]`
- 這行在提取BERT類模型的句子表示向量：
  - `last_hidden_state` 的形狀是 `[batch_size, sequence_length, hidden_size]`
  - `[:, 0, :]` 這個切片操作代表：
    - 第一個 `:` : 選取所有batch
    - `0` : 只取第一個token位置(即[CLS]標記)
    - 最後的 `:` : 選取所有hidden維度
  - [CLS]標記通常被用來表示整個句子的語義

2. `embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)`
- 這行在進行L2正規化：
  - `p=2` : 使用L2範數(歐幾里得範數)
  - `dim=1` : 在hidden維度上進行正規化
  - 正規化後的向量長度會變為1，這有助於：
    - 計算向量間的相似度
    - 減少向量大小的影響
    - 提高模型的穩定性

簡單來說，這兩行程式碼的目的是：
1. 獲取能夠代表整個輸入文本意義的向量表示
2. 對向量進行標準化處理，使其更適合後續的相似度計算

## 說明2
這兩種取得句向量(sentence embeddings)的方法確實有所不同：

### 您先前的方法：


In [ ]:
embeddings = outputs.last_hidden_state[:, 0, :] #使用[CLS]token的輸出
embeddings = torch.nn.functional.normalize(embeddings, p=2, dim=1)



### 目前的方法：


In [ ]:
sentence_embeddings = outputs.last_hidden_state.mean(dim=1)



主要差異在於：

1. **向量提取方式**
   - 先前方法：只使用 `[CLS]` token 的隱藏狀態作為整個句子的表示
   - 目前方法：對所有 tokens 的隱藏狀態取平均值

2. **正規化處理**
   - 先前方法：有進行 L2 正規化
   - 目前方法：沒有進行正規化

3. **理論基礎**
   - `[CLS]` token：BERT 類模型特別設計用來捕捉整個句子的語義信息
   - 平均池化：考慮了所有 tokens 的貢獻，可能包含更多局部細節

### 建議改進
建議您使用先前的方法，因為：
1. `[CLS]` token 是模型專門設計來表示整個句子的
2. L2 正規化有助於提高相似度計算的穩定性
3. 這也是 E5 模型原始論文中推薦的方法

可以這樣修改當前代碼：


In [ ]:
#提取句子嵌入
sentence_embeddings = outputs.last_hidden_state[:, 0, :]  # 使用[CLS]token
sentence_embeddings = torch.nn.functional.normalize(sentence_embeddings, p=2, dim=1)

## 說明3
讓我解釋 `embeddings.cpu().numpy()[0]` 這行代碼的每個部分：

1. **embeddings**: 
   - 這是一個在 GPU 或 CPU 上的 PyTorch tensor
   - 包含了經過模型處理後的向量表示

2. **.cpu()**:
   - 將 tensor 從 GPU 移動到 CPU
   - 如果原本就在 CPU 上，這個操作不會有影響
   - 這個步驟是必要的，因為 numpy 只能處理 CPU 上的數據

3. **.numpy()**:
   - 將 PyTorch tensor 轉換為 NumPy array
   - NumPy array 在許多數據處理任務中更容易使用

4. **[0]**:
   - 選取第一個（也是唯一一個）樣本的嵌入向量
   - 因為模型輸出的形狀是 `[1, embedding_dim]`（批次大小為1）
   - 這個操作移除了第一個維度，得到一個一維數組

完整流程：


In [ ]:
PyTorch Tensor (GPU/CPU) 
→ CPU Tensor 
→ NumPy Array 
→ 一維向量



這個轉換是必要的，因為：
- NumPy 格式更適合後續的數值計算
- 移除批次維度使向量更容易處理
- 確保數據在 CPU 上，可以進行更多標準的數據操作

## 說明4



In [ ]:
similarities = np.dot(np.stack(df['Embeddings'].values), query_embedding)



### 程式碼拆解說明：

1. **`df['Embeddings'].values`**
   - 取得 DataFrame 中 'Embeddings' 欄位的所有向量值
   - 每個向量代表一個文件的嵌入表示

2. **`np.stack()`**
   - 將多個嵌入向量堆疊成一個二維數組
   - 結果形狀：`[文件數量, 嵌入維度]`

3. **`np.dot(文件矩陣, 查詢向量)`**
   - 計算文件矩陣和查詢向量的點積
   - 等同於計算每個文件向量與查詢向量的餘弦相似度
   - 因為向量已經經過 L2 正規化，點積結果就等於餘弦相似度

### 舉例說明：
如果有：
- 2 個文件
- 嵌入維度是 3
- 一個查詢向量

則運算過程為：


In [ ]:
文件矩陣: [[0.1, 0.2, 0.3],    點積    查詢向量: [0.4, 0.5, 0.6]
          [0.7, 0.8, 0.9]]     ==>    相似度: [0.32, 0.77]



### 輸出結果：
- 返回一個一維數組，長度等於文件數量
- 每個數值代表對應文件與查詢的相似度分數
- 分數越高表示越相關